# Xsarsea example

The Normalized Radar Cross Section (sigma0) as computed from Level-1 SAR data can be detrended in the case of ocean scenes. 

The goal is to remove the averaged trend (decreasing) of the NRCS with (increasing) incidence angle observed for acquisitions over ocean. 

The detrend maximizes the contrasts in the image due to geophysical phenomena and improves the visualization experience of ocean scenes. 

**Sigma0_detrend** is also termed **image roughness** or **nice display**.

In [ ]:
import xsar

import xsarsea

In [ ]:
# use holoviews for plots
import bokeh.io

bokeh.io.output_notebook()
import holoviews as hv

hv.extension('bokeh')

In [ ]:
# optional debug message
import logging

logging.basicConfig()
logging.getLogger('xsarsea').setLevel(logging.INFO) # .setLevel(logging.DEBUG) for more messages

## read the dataset with xsar

In [ ]:
# get test file. You can replace with an path to other SAFE
filename = xsar.get_test_file('S1A_IW_GRDH_1SDV_20170907T103020_20170907T103045_018268_01EB76_Z010.SAFE')
filename

In [ ]:
# open the dataset with xsar
sar_ds = xsar.open_dataset(filename, resolution='1000m')
sar_ds[['longitude','latitude','sigma0','incidence']]


## Sigma0 detrending with xsarsea

Sigma0 detrending is done by  [xsarsea.sigma0_detrend](../basic_api.rst#xsarsea.sigma0_detrend) function

As the resulting xarray dataset have the same coordinates as the original sigma0, we can add a `sigma0_detrend` variable to the dataset.


By default, analytical models are available. We load all luts : 
<span style="color:red">replace paths by your own path containing all luts</span>




In [ ]:
nc_luts_path = xsarsea.get_test_file('nc_luts_reduce')
path_cmod7 = xsarsea.get_test_file("cmod7_and_python_script")
xsarsea.windspeed.register_luts(nc_luts_path, path_cmod7)
xsarsea.windspeed.available_models()

### default detrend with gmf_cmod5n

In [ ]:
sar_ds['sigma0'] = sar_ds.sigma0
sar_ds['sigma0_detrend'] = xsarsea.sigma0_detrend(sar_ds.sigma0, sar_ds.incidence)

In [ ]:
original_sigma0_plot = hv.Image(sar_ds.sigma0.sel(pol='VV')).opts(
    title="Original Sigma0",
    cmap='gray',
    clim=(0, 0.4),
    colorbar=True,
    width=400,
    height=400,
    tools=['hover']
)

detrended_sigma0_plot = hv.Image(sar_ds.sigma0_detrend.isel(pol=0)).opts(
    title="Detrended Sigma0",
    cmap='gray',
    clim=(0, 0.4),
    colorbar=True,
    width=400,
    height=400,
    tools=['hover']
)

(original_sigma0_plot + detrended_sigma0_plot).cols(2)

### custom detrend

very small differences

In [ ]:
detrend_vv_gmf_cmod7 = xsarsea.sigma0_detrend(sar_ds.sigma0.sel(pol='VV'), sar_ds.incidence, model='gmf_cmod7')
detrend_vv_gmf_cmod5 = xsarsea.sigma0_detrend(sar_ds.sigma0.sel(pol='VV'), sar_ds.incidence, model='gmf_cmod5')
detrend_vv_gmf_cmod5n = xsarsea.sigma0_detrend(sar_ds.sigma0.sel(pol='VV'), sar_ds.incidence, model='gmf_cmod5n')
## HH GMF
detrend_vv_gmf_cmod5n_zhangA = xsarsea.sigma0_detrend(sar_ds.sigma0.sel(pol='VV'), sar_ds.incidence, model='cmod5n_Rlow_hh_zhangA')


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axs = plt.subplots(1, 4, figsize=(20, 5))

datasets = [detrend_vv_gmf_cmod7, detrend_vv_gmf_cmod5, detrend_vv_gmf_cmod5n, detrend_vv_gmf_cmod5n_zhangA]
titles = ['GMF CMOD7', 'GMF CMOD5', 'GMF CMOD5N', 'GMF CMOD5N ZhangA']

for ax, data, title in zip(axs, datasets, titles):
    img = ax.pcolormesh(sar_ds.longitude, sar_ds.latitude, data, cmap='gray', vmin=0, vmax=0.4)
    ax.set_title(title)
    ax.set_xlabel("longitude")
    ax.set_ylabel("latitude")
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(img, cax=cax)

# Improve spacing between plots
plt.tight_layout()
plt.show()
